In [18]:
import ifcopenshell
import time

#INPUT:
in_path = r"C:\Users\eibre\OneDrive - Norconsult Group\Documents\VOS-RIBarmering_ifc4.ifc"
out_path = in_path[:-4]+"_color.ifc"
pset_name = 'IFC Armering'
property_name = '11 PosNr'
colours = [
    [255, 255, 0], [0, 255, 255], [255, 0, 255], [0, 0, 255], [255, 153, 0], [255, 0, 0], [0, 204, 255], [153, 204, 0], [255, 102, 0], [255, 0, 255], [128, 0, 0], [128, 128, 0], [0, 128, 0], [0, 128, 128], [0, 0, 128], [128, 0, 128], [255, 128, 128], [255, 255, 153], [51, 153, 102], [153, 204, 255], [255, 153, 0], [51, 102, 255], [51, 153, 102], [255, 204, 153], [255, 255, 0], [0, 0, 255], [153, 153, 255], [51, 204, 204], [128, 0, 128], [0, 128, 128], [128, 128, 0], [255, 128, 128], [51, 102, 255], [255, 153, 204], [255, 204, 153], [128, 0, 128], [0, 128, 0], [255, 204, 0], [51, 102, 255], [51, 102, 255], [0, 255, 0], [153, 153, 255], [255, 255, 0], [255, 0, 255], [51, 204, 204], [51, 102, 255], [204, 204, 255], [51, 102, 255], [204, 255, 204], [255, 0, 0], [0, 0, 255], [51, 153, 102], [255, 255, 0], [255, 153, 0], [153, 204, 255], [255, 102, 0], [204, 153, 255], [204, 204, 255], [255, 153, 204], [51, 51, 153], [255, 255, 0], [0, 0, 255], [153, 153, 255], [51, 204, 204], [128, 0, 128], [0, 128, 128], [128, 128, 0], [255, 128, 128], [51, 102, 255], [255, 153, 204], [255, 0, 0], [0, 0, 255], [51, 153, 102], [255, 255, 0], [255, 153, 0], [153, 204, 255], [255, 102, 0], [204, 153, 255], [204, 204, 255], [255, 153, 204]
]

# Script starts:
t_start = time.perf_counter()

print(f"Opening file at {in_path}")

f = ifcopenshell.open(in_path)
rebars = f.by_type('IfcReinforcingBar')
print(f'Modellen inneholder {len(rebars)} rebars')
IfcSweptDiskSolid = f.by_type('IfcSweptDiskSolid')
print(f'Modellen inneholder {len(IfcSweptDiskSolid)} IfcSweptDiskSolid')
styleItems = f.by_type('IfcStyledItem')
print(f'Modellen inneholder {len(styleItems)} styled items')

styles = f.by_type('IfcSurfaceStyle')
print(f'Modellen inneholder {len(styles)} styles:')

for style in styles:
    print(f' * {style.Name}')
def get_propery_value(rebar, pset_name, p_name):
    for pset in rebar.IsDefinedBy:
        if pset.RelatingPropertyDefinition.Name == pset_name:
            for property in pset.RelatingPropertyDefinition.HasProperties:
                if property.Name == p_name:
                    return property.NominalValue[0]

import collections
from collections import defaultdict
rebar_dict = defaultdict(list)
print(f"Grouping rebar by values of property {property_name} in pset {pset_name}")
for k,v in zip([int(get_propery_value(rebar, pset_name, property_name)) for rebar in rebars], rebars):
    rebar_dict[k].append(v)
    
ordered_rebars = collections.OrderedDict(sorted(rebar_dict.items()))
num_groups = len(ordered_rebars)
print(f"Found {num_groups} different groups")

import math
colours = colours*math.ceil(num_groups/len(colours))
colours = colours[:num_groups]
rgb_colours = list()

for rgb in colours:
    rgb_colours.append(f.createIfcColourRgb(None, rgb[0]/255, rgb[1]/255, rgb[2]/255))
if(len(styleItems) > 10):
    print("Removing existing IfcStyledItems")
    for styleItem in styleItems:
        f.remove(styleItem)

i=0
print("Creating styled items for rebar color..")
for k, group in ordered_rebars.items():
    colour = rgb_colours[i]
    print(f"Group where {property_name} = {k} contains {len(group)} elements")
    surfaceStyleShading = f.createIfcSurfaceStyleShading()
    surfaceStyleShading.SurfaceColour = colour
    surfaceStyle = f.createIfcSurfaceStyle(colour.Name, "BOTH",(surfaceStyleShading,))
    presStyleAssign = f.createIfcPresentationStyleAssignment((surfaceStyle,))
    i=i+1
    for rebar in group:
        item = rebar.Representation.Representations[0].Items[0]
        f.createIfcStyledItem(item, (presStyleAssign,), None)
        
print(f"Saving file to {out_path}..")
f.write(out_path)
t_end = time.perf_counter()
print(f"File saved, script completed in in {t_end-t_start:0.4f} seconds")

# print("Running Solibri Optimizer..")
# import subprocess
# dir_path = r"X:\nor\oppdrag\Trondheim\520\31\5203127\BIM\Konstruksjon\IFC"
# subprocess.check_call([r'C:\Program Files\Solibri\IFCOptimizer\Solibri IFC Optimizer.exe', '-in=' + in_path, '-out=' + dir_path, '-ifc', '-force'])
 
# import os
# file_clean_name = dir_path+"\\"+os.path.basename(in_path)
# file_optimized = file_clean_name[:-4]+"_optimized.ifc"
# print(f"Renaming {file_optimized} to {file_clean_name}")
# os.replace(file_optimized, file_clean_name)
# print("Done!")

Opening file at C:\Users\eibre\OneDrive - Norconsult Group\Documents\VOS-RIBarmering_ifc4.ifc
Modellen inneholder 56774 rebars
Modellen inneholder 57123 IfcSweptDiskSolid
Modellen inneholder 57278 styled items
Modellen inneholder 4 styles:
 * B500NC
 * NO_Metall - Stål - S355
 * Iron B500 NC
 * Steel, Wrought
Grouping rebar by values of property 11 PosNr in pset IFC Armering
Found 99 different groups
Removing existing IfcStyledItems
Creating styled items for rebar color..
Group where 11 PosNr = 1 contains 9320 elements
Group where 11 PosNr = 2 contains 5073 elements
Group where 11 PosNr = 3 contains 4691 elements
Group where 11 PosNr = 4 contains 3325 elements
Group where 11 PosNr = 5 contains 2734 elements
Group where 11 PosNr = 6 contains 1687 elements
Group where 11 PosNr = 7 contains 2162 elements
Group where 11 PosNr = 8 contains 2620 elements
Group where 11 PosNr = 9 contains 1960 elements
Group where 11 PosNr = 10 contains 1624 elements
Group where 11 PosNr = 11 contains 999 ele

In [2]:
import ifcopenshell
in_path = r"C:\Users\eibre\OneDrive - Norconsult Group\Documents\VOS-RIBarmering_ifc4.ifc"
out_path = in_path
f = ifcopenshell.open(in_path)

In [13]:
rebars = f.by_type('IfcReinforcingBar')
print(f'Modellen inneholder {len(rebars)} rebars')
IfcSweptDiskSolid = f.by_type('IfcSweptDiskSolid')
print(f'Modellen inneholder {len(IfcSweptDiskSolid)} IfcSweptDiskSolid')
styleItems = f.by_type('IfcStyledItem')
print(f'Modellen inneholder {len(styleItems)} styled items')

styles = f.by_type('IfcSurfaceStyle')
print(f'Modellen inneholder {len(styles)} styles:')

for style in styles:
    print(f' * {style.Name}')

Modellen inneholder 56774 rebars
Modellen inneholder 57123 IfcSweptDiskSolid
Modellen inneholder 57278 styled items
Modellen inneholder 4 styles:
 * B500NC
 * NO_Metall - Stål - S355
 * Iron B500 NC
 * Steel, Wrought


In [16]:
in_path = r"C:\Users\eibre\OneDrive - Norconsult Group\Documents\VOS-RIBarmering_ifc4.ifc"
out_path = in_path[:-4]+"_color.ifc"
print(out_path)

C:\Users\eibre\OneDrive - Norconsult Group\Documents\VOS-RIBarmering_ifc4_color.ifc
